# julia
# C関数の呼び出し
PythonやMATLABよりは簡単に呼び出せる． `ccall`を使う．  

### 流れ  
1. C言語で呼び出したい関数のソースコードを作る
2. 作ったソースコードを共有ライブラリ（.so形式）としてビルドする
3. juliaで共有ライブラリを呼び出す関数（ラッパー）を書く
4. 終わり

## 簡単な例
シグモイド関数を作ってみる．  
$$
\zeta(x) = \frac{1}{1+e^{-ax}}
$$
以下C言語のソース  
**sigmoid.c**  
```c
#include <math.h>
double sigmoid(double x, double a)
{
    return 1 / (1 + exp(-a*x));
}
```

共有ライブラリとしてビルドする．  
```bash
gcc -shared -fPIC -o sigmoid.so sigmoid.c
```

共有ライブラリ`sigmoid.so`が作成される．  
最後に`ccall`を使ってラッパーを書く．  

```julia
ccall(
    (:呼びたい関数名, 共有ライブラリのパス),
    戻り値の型,
    (引数の型A, 引数の型B, ...),
    引数A, 引数B, ...
)
```

In [1]:
function sigmoid(x::Float64, a::Float64)
    ccall(
        (:sigmoid, "sigmoid.so"),
        Cdouble,
        (Cdouble, Cdouble),
        x, a
    )
end

sigmoid (generic function with 1 method)

普通の関数と同じように利用できる．  

In [11]:
sigmoid(1., 2.)

0.8807970779778823

C言語の型とjuliaの型の対応関係は以下の通り．  
|C|Alias|julia|
|----|----|----|
|void|Cvoid|Cvoid|
|int|Cint|Int32|
|doube|Cdouble|Float64|

[https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/#mapping-c-types-to-julia](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/#mapping-c-types-to-julia)

0.8807970779778823